In [ ]:
using CSV, DataFrames, Statistics

In [ ]:
filelist = [ f for f in split.(read(`ls ../results/penalty`, String), "\n") if endswith(f, ".csv") && !startswith(f, "0_")]

In [ ]:
df = DataFrame(CSV.read("../results/penalty/"*filelist[1]));

In [ ]:
for i in 3:length(filelist)
    aux = DataFrame(CSV.read("../results/penalty/"*filelist[i]))
    df = vcat(df, aux[names(df)])
end

In [ ]:
function iqr(t)
    try
        return quantile(t, .75) - quantile(t, .25)
    catch 
        return NaN
    end
end

In [ ]:
res = aggregate(df[[:dataset, :method, :penalty, :osr2]], [:dataset, :method, :penalty], [mean, median, std, iqr])

In [ ]:
CSV.write("../results/penalty/0_summary.csv", res)

In [ ]:
using  Plots, StatsPlots

In [ ]:
filter!( t -> t[:dataset] ∉ ["hepatitis", "trains"], df);

In [ ]:
widedf = unstack(df, [:dataset, :copynum, :penalty, :iter], :method, :osr2)
widedf[!,:augmented_improvement] = (widedf[!,Symbol("Augmented")] .- widedf[!,Symbol("Impute then regress")]) ./ abs.(widedf[!,Symbol("Impute then regress")])
widedf[!,:affine_improvement] = (widedf[!,Symbol("Augmented Affine")] .- widedf[!,Symbol("Impute then regress")]) ./ abs.(widedf[!,Symbol("Impute then regress")])

In [ ]:
filter!(t -> t[:augmented_improvement] < 1, widedf)
filter!(t -> t[:augmented_improvement] > -1, widedf)
filter!(t -> t[:affine_improvement] < 1, widedf)
filter!(t -> t[:affine_improvement] > -1, widedf);

In [ ]:
@df widedf density(:augmented_improvement, label="Augmented")
@df widedf density!(:affine_improvement, label="Affine")

In [ ]:
wide_res = aggregate(widedf[[:dataset, :penalty, :augmented_improvement, :affine_improvement]],
    [:dataset, :penalty], [mean, median, std, iqr]);

In [ ]:
@df wide_res density(:augmented_improvement_mean, label="Augmented")
@df wide_res density!(:affine_improvement_mean, label="Affine")

In [ ]:
savefig("plot.png")